### First study EIXUZQ


### Summary of notebook: 

This dataset gives the information of the diagnostic (FinalStatus) and the prognostic (TypeOfExit) of each patient. 


*Data analysis*

- First we looked at the patients and we found out that one patient has two records. We assumed that is was another patient (second patient F716 become patient F716_bis)

- Then we looked at the distributions of certain characteristics of the patients (age, sex, pregnant, HealthCareWorker, Occupation, County), and at the chronology of the epidemic (distribution of the daily start of illness reported, distribution of the monthly start report, etc). We noticed that there was no information about the pregnancy of the patients (only nans) so we dropped this column. 

- We computed the time between the start of the illness and the report. This allowed us to detect that a patient was reported before his illness began. So we removed it.

- Thanks to the previous calculation of the reference time, we were able to draw the distribution of the number of cases diagnosed according to the reference time. We did the same for the outcome. 
We were also interested in whether the waiting time before admission to the clinic, and the time spent at the clinic, had an effect on patient outcome. 

- We finally took a quick look at the contacts of the patients, and at the results of the malaria tests as we are aware that the symptoms are close to those of ebola. 


*Processing*

- For the final choice of dataframe on which we will build our models, we decided to keep only certain information about the patient (sex, age, Referraltime) as well as the outputs (FinalStatus and TypeofExit). 

- This choice of dataframe is followed by a bit of cleaning to remove the patients with no known symptoms (we drop these rows).

- To fill in the missing values, we made the (debatable) assumption that when the symptom is not filled in, it means that the patient does not present this symptom (we fill the Nan with zeros). 


*Data visualization*

- A visualization of the correlation matrix confirms that our features are not overly correlated. 

- Then we identified the features with the highest correlations with the target value (FinalStatus). 

- We checked the balance of our dataset and we concluded that our dataset is quite unbalanced and that it will have to be taken into account in the interpretation of the results (especially for accuracy).

- Finally, we tried to identify clusters in our data, using a function that shows how strongly each feature influences a principal component (PCA1 or PCA2). But unfortunately no cluster was detected in either two or three dimensions.

So we split our data to start with the models, checking that the folds preserve the percentage of samples for both class (stratify attribute).


*Models*

The list of performed models is as follows:

1. Least-squares

2. Logistic 

For both first models we performed backward elimination and then updated the models with the selected features. 
We also performed recursive feature elimination for linear and logistic regression (ex: for logistic regression the optimum number of features was 11). 
Finally we tried some Ensemble methods (see the description in the corresponding cell). This gave us the feature importance with a lasso model. 

3. Decision Trees

4. Random Forest 

For the last two models we have obtained graphic illustrations.

Our latest model, XGBoost, is the most complex model in terms of the number of parameters. A description of the model is given in the corresponding cells.  

5. XGBoost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from proj2_HELPERS_ import *
from yellowbrick.model_selection import ValidationCurve
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


# Tool to calculate carbon footprint
from cumulator import base

cumulator = base.Cumulator()
cumulator.on()

In [2]:
# Enter path to cleaned data
df = pd.read_excel('../../Cleaned_data/EIXUZQ_LIB_FOYA.xls', header =[1])

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../Cleaned_data/EIXUZQ_LIB_FOYA.xls'

In [ ]:
df["FinalStatus"].value_counts(dropna=False)


In [ ]:
df.FinalStatus.value_counts(dropna=False).plot(kind='pie',labels=['Confirmed', 'Not a case', 'Probable', 'Nan'], colors=['pink', 'b', 'c', 'r'], fontsize=10, figsize=(6, 6))

In [ ]:
df.info()

In [ ]:
#sns.heatmap(df.isnull(), cbar=False)


f, ax = plt.subplots(figsize=(14, 7))
ax = sns.heatmap(df.isnull(), cbar=False)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 0)
plt.savefig('Data_distribution_1.png')

### Numero

In [ ]:
sum(df['Sex'].isna())

In [ ]:
df['numero'].nunique()

One numero has 2 records. 

In [ ]:
df['numero'].value_counts(dropna=False)

The patient with two records is F716.

In [ ]:
df[df['numero'] == 'F716']

In [ ]:
df['Age']

In [ ]:
df.loc[849, 'numero'] = 't_bis'

### Sex

In [ ]:
sum(df['Sex'].isna())

In [ ]:
df['Sex'].value_counts(dropna=False)

In [ ]:
dict_sex = {'m' : 'M', 'f' : 'F'}

def correct_sex(row):
    if row.Sex in dict_sex:
            return dict_sex[row.Sex]
    return row.Sex

df['Sex'] = df.apply(correct_sex,axis = 1)

In [ ]:
df['Sex'].value_counts(dropna=False)

### Age

In [ ]:
sum(df['Age'].isna())

In [ ]:
df['Age']

In [ ]:
df['Age'].mean()

In [ ]:
df['Age'].median()

In [ ]:
df['Age'].min()

In [ ]:
df['Age'].max()

In [ ]:
df['Age'].value_counts(dropna=False)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.set_title("Boxplot of the age",size=15)
plt.boxplot(df.loc[~df['Age'].isna(),'Age'])
plt.show()

### Pregnant

In [ ]:
sum(~df['Pregnant'].isna())

In [ ]:
df.drop(columns= ['Pregnant'], inplace = True)

### HealthCareWorker

In [ ]:
df['HealthCareWorker'].value_counts(dropna=False)

### Occupation

In [ ]:
sum(df['Occupation'].isna())

In [ ]:
df['Occupation'] = df['Occupation'] .str.lower()
df['Occupation'].value_counts(dropna=False).head()

### County

In [ ]:
sum(df['County'].isna())

In [ ]:
df['County'] = df['County'].str.lower()
df['County'].value_counts(dropna=False)

### DateIllnessStarted

In [ ]:
sum(df['DateIllnessStarted'].isna())

In [ ]:
df['DateIllnessStarted_day'] = df['DateIllnessStarted'].dt.to_period('D')
df['DateIllnessStarted_month'] = df['DateIllnessStarted'].dt.to_period('M')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

df['DateIllnessStarted_day'].value_counts().sort_index().plot(kind ="bar")

ax.set_title("Distribution of the daily start of illness reported",size=15)
ax.set_xlabel('Day')
ax.set_ylabel('Number of case')
ax.xaxis.set_major_locator(plt.MaxNLocator(30))
plt.xticks(rotation=70)
plt.show()

### DateofCaseReport

In [ ]:
sum(df['dateofCaseReport'].isna())

In [ ]:
df['dateofCaseReport_day'] = df['dateofCaseReport'].dt.to_period('D')
df['dateofCaseReport_month'] = df['dateofCaseReport'].dt.to_period('M')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

df['dateofCaseReport_day'].value_counts(dropna=False).sort_index().plot(kind ="bar")

ax.set_title("Distribution of the daily case report",size=15)
ax.set_xlabel('Day')
ax.set_ylabel('Number of case report')
ax.xaxis.set_major_locator(plt.MaxNLocator(30))
plt.xticks(rotation=70)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

df['dateofCaseReport_month'].value_counts(dropna=False).sort_index().plot(kind ="bar")

ax.set_title("Distribution of the monthly case report",size=15)
ax.set_xlabel('Month')
ax.set_ylabel('Number of case report')
#ax.xaxis.set_major_locator(plt.MaxNLocator(30))
plt.xticks(rotation=70)
plt.show()

### Time between start of sickness and report

In [ ]:
def compute_diff_days(row):
    if pd.isnull(row.DateIllnessStarted):
        return np.nan
    else: 
        return (row.dateofCaseReport - row.DateIllnessStarted)

In [ ]:
df['Referraltime'] = df[['DateIllnessStarted','dateofCaseReport']].apply(compute_diff_days, axis = 1 )

In [ ]:
df['Referraltime'] 

In [ ]:
sum(df['Referraltime'].isna())

In [ ]:
df['Referraltime'] .value_counts()

In [ ]:
df[df['Referraltime'] == '-29 days']

We have an odd value and we decide to drop it. 

In [ ]:
df.drop(index = 762, inplace = True)

In [ ]:
df_dif_status = df.groupby(['Referraltime','FinalStatus']).numero.count().unstack(fill_value=0)
df_dif_status.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_dif_status.plot(kind='bar',stacked=True, ax=ax, color = ['red', 'royalblue', 'brown'])

plt.legend(loc="lower left", bbox_to_anchor=(1,0))
ax.set_title("Distirbution of the referral time",size=14)
ax.set_xlabel('Day waited to report')
ax.set_ylabel('Number of individuals')
plt.show()

In [ ]:
df_dif_exit = df.groupby(['Referraltime','TypeOfExit']).numero.count().unstack(fill_value=0)
df_dif_exit.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_dif_exit.plot(kind='bar',stacked=True, ax=ax, color = ['red', 'royalblue', 'brown'])

plt.legend(loc="lower left", bbox_to_anchor=(1,0))
ax.set_title("Distirbution of the referral time",size=14)
ax.set_xlabel('Day waited to report')
ax.set_ylabel('Number of individuals')
plt.show()

### readmission

In [ ]:
df['readmission'].value_counts(dropna=False)

### DelayBeforeAdmission

In [ ]:
df_delay_case = df.groupby(['DelayBeforeAdmission','FinalStatus']).numero.count().unstack(fill_value=0)
df_delay_case.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_delay_case.plot(kind='bar',stacked=True, ax=ax, color = ['red', 'royalblue', 'brown'])

plt.legend(loc="lower left", bbox_to_anchor=(1,0))
ax.set_title("Distirbution of the delay before admission",size=14)
ax.set_xlabel('Day delayed')
ax.set_ylabel('Number of individuals')
plt.show()

In [ ]:
df['TypeOfExit'] = df['TypeOfExit'].str.lower()

In [ ]:
df_delay_exit = df.groupby(['DelayBeforeAdmission','TypeOfExit']).numero.count().unstack(fill_value=0)
df_delay_exit.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_delay_exit.plot(kind='bar',stacked=True, ax=ax, color = ['green', 'red','royalblue', 'brown'])

plt.legend(loc="lower left", bbox_to_anchor=(1,0))
ax.set_title("Distirbution of the delay before admission",size=14)
ax.set_xlabel('Day delayed')
ax.set_ylabel('Number of individuals')
plt.show()

In [ ]:
df_delay_exit['Mortality rate'] = df_delay_exit.apply(lambda row : row['died'] / sum(row), axis = 1)

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))

delay_mr = df_delay_exit.loc[:7,'Mortality rate']

plt.scatter(delay_mr.index, delay_mr)
ax.set_title("Mortality rate per delay before admission",size=14)
ax.set_xlabel('Day delayed')
ax.set_ylabel('Mortality rate')
ax.set_ylim(0, 1) 
plt.show()

### TypeOfExit

In [ ]:
sum(df['TypeOfExit'].isna())

In [ ]:
df.loc[df['TypeOfExit'].isna(), 'FinalStatus'].value_counts(dropna=False)

In [ ]:
df['TypeOfExit'] = df['TypeOfExit'].fillna('unknown')

In [ ]:
df['TypeOfExit'].value_counts(dropna=False)

### FinalStatus

In [ ]:
df['FinalStatus'].value_counts(dropna=False)

In [ ]:
pd.DataFrame(df.groupby(['FinalStatus','TypeOfExit']).numero.count())

### LenghtOfStay 

In [ ]:
sum(df['LenghtOfStay'].isna())

In [ ]:
df_lstay_case = df.groupby(['LenghtOfStay','FinalStatus']).numero.count().unstack(fill_value=0)
df_lstay_case.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_lstay_case.plot(kind='bar',stacked=True, ax=ax, color = ['red', 'royalblue', 'brown'])

plt.legend(loc="lower left", bbox_to_anchor=(1,0))
ax.set_title("Distirbution of the length of stay",size=14)
ax.set_xlabel('Day stayed')
ax.set_ylabel('Number of individuals')
plt.show()

In [ ]:
df_lstay_exit = df.groupby(['LenghtOfStay','TypeOfExit']).numero.count().unstack(fill_value=0)
df_lstay_exit.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_lstay_exit.plot(kind='bar',stacked=True, ax=ax, color = ['green', 'red', 'brown', 'royalblue', 'yellow'])

plt.legend(loc="lower left", bbox_to_anchor=(1,0))
ax.set_title("Distirbution of the length of stay",size=14)
ax.set_xlabel('Day stayed')
ax.set_ylabel('Number of individuals')
plt.show()

### Test Result

In [ ]:
df['First'].value_counts(dropna=False)

In [ ]:
dict_result = {'P' : 'Positive', 'N' : 'Negative', 'I' : 'Inconclusive', np.nan : 'Not done'}

In [ ]:
df['First'] = df['First'].apply(lambda x: dict_result[x])
df['Second'] = df['Second'].apply(lambda x: dict_result[x])
df['Third'] = df['Third'].apply(lambda x: dict_result[x])

In [ ]:
df_tests_outcome = pd.DataFrame(df.groupby(['First','Second','Third', 'TypeOfExit','FinalStatus']).numero.count())
df_tests_outcome

### MalariaTest

In [ ]:
df['MalariaTest'] = df['MalariaTest'].str.lower()
df['MalariaTest'].value_counts(dropna=False)

In [ ]:
dict_malaria = {'negative' : 'Negative', 'not done' : 'Unknown', 'positive' : 'Positive', 
                'n/a' : 'Unknown', 'no info' : 'Unknown', 'inconclusive' : 'Unknown', 
                'postive' : 'Positive', np.nan : 'Unknown'}

df['MalariaTest'] = df['MalariaTest'].apply(lambda x: dict_malaria[x])
df['MalariaTest'].value_counts(dropna=False)

In [ ]:
pd.DataFrame(df.groupby(['MalariaTest', 'FinalStatus']).numero.count())

### TypeOfExit 

In [ ]:
df['TypeOfExit'].value_counts(dropna=False)

### Contact

In [ ]:
contacts = ['Someone ill in the familiy','Visited someone ill','somebody recently died in your family','been to a funeral recently']

In [ ]:
df.loc[:,contacts]=df.loc[:,contacts].fillna('N')

In [ ]:
df['Someone ill in the familiy'].value_counts(dropna=False)

In [ ]:
df['Visited someone ill'].value_counts(dropna=False)

In [ ]:
df['somebody recently died in your family'].value_counts()

In [ ]:
df['been to a funeral recently'].value_counts(dropna=False)

In [ ]:
df.loc[:,contacts]=df.loc[:,contacts].applymap(lambda x: 1 if x == 'Y' else 0)

In [ ]:
df.info()

### Details of type of contact with FHF patient

details_contact = ['Yes/No', 'Slept in the same house', 'Had direct physical contact', 'Touched their body fluids', 'had sexual relations',  
'Handled clothes or other personal objetc']              


### Symptoms since illness started

In [ ]:
symptoms = ['fever', 'Vomit', 'Nausea', 'Diarrhoea', 'AstheniaWeakness', 'LossOfAppetite', 'AbdominalPain', 'ChestPain', 
'BoneMusclePain', 'JointPain','Headache', 'Cough', 'Breathlessness', 'SwallowingProblem',
'Sorethroat', 'Jaundice', 'Conjunctivitis', 'HemoragicEyes','SkinRash', 'Hichups', 'PainEyesSensitivityLight',
'Coma', 'ConfusedDisoriented', 'OtherHaemorraghe']

infos = ['Sex', 'Age', 'Referraltime']
tests = ['CT Values', 'MalariaTest']
outputs = ['FinalStatus', 'TypeOfExit']

In [ ]:
df_ml = df[infos+ symptoms + tests + outputs].copy()
df_ml.head()

In [ ]:
df_ml

We drop the row that have only Nan values in the symptoms. (870 - 511)

In [ ]:
df_ml.dropna(how='all', subset = symptoms)

In [ ]:
df_ml.dropna(how='all', subset = symptoms, inplace = True) 
df_ml[symptoms] = df_ml[symptoms].fillna('N')
dict_binary_symptoms = {'Y' : 1, 'N' : 0}
df_ml[symptoms] = df_ml[symptoms].applymap(lambda x: int(dict_binary_symptoms[x]))
df_ml.head()

In [ ]:
df_ml[symptoms+outputs].groupby(outputs).sum()

# ML models

In [ ]:
df_ml['Sex'] = df_ml['Sex'].apply(lambda x: 1 if x == 'M' else 0)

In [ ]:
df_ml['Referraltime'] = df_ml['Referraltime'].apply(lambda x: x.days)

**Setting 1 Triage :** Using only the personnal informations and the symptoms. We want to find out if the patient has Ebola (FinalStatus)

In [ ]:
from sklearn.preprocessing import StandardScaler

#select the features needed
df_triage = df_ml[infos+ symptoms+ ['FinalStatus']]
df_triage = df_triage[(df_triage['FinalStatus'] != 'Probable') & ~(df_triage['FinalStatus'].isna())]

#transform the dependent variable
dict_fstatus = {'Confirmed' : 1, 'Not a Case' : 0}
df_triage['FinalStatus'] = df_triage['FinalStatus'].apply(lambda x : dict_fstatus[x]) 
df_triage.dropna(how='any',inplace= True)


In [ ]:
X_ebo_ml = df_triage.drop(columns="FinalStatus")
y_ebo_ml = df_triage['FinalStatus']

## Data visualization

### Correlation

In [ ]:
Corr_vision(X_ebo_ml)

In [ ]:
## Correlation with target label 

X_y = X_ebo_ml.join(y_ebo_ml)
corr_matrix = X_y.corr()

corr_y = corr_matrix['FinalStatus']
threshold = 0.2
fig, ax = plt.subplots(figsize=(10,6))
corr_y[corr_y.index[abs(corr_y) > threshold].tolist()].drop('FinalStatus').plot(kind='barh')
plt.title('Features that have an absolute pearson correlation value superior to {} with target variable'.format(threshold))
plt.show()

The values above are "correlated" with the output variable 'Finalstatus' (ebola outcome), we expect them to be considered important during the model.

### Class imbalance

Balance is important in order to get a reliable accuracy for unseen datas, if imbalanced, steps need to be taken in order to take this into account. 
A good metric to look at is precision, recall and F1, this is discussed in the report. 

In [ ]:
Imbalance(y_ebo_ml)

In [ ]:
Rad_vision(X_ebo_ml, y_ebo_ml)

Looking at the figure above, we get no real difference between ebola negative and ebola positive patients.

In [ ]:
PCA_vision_3D(X_ebo_ml, y_ebo_ml)

In [ ]:
from sklearn.decomposition import PCA
k = 2 # Number of components
pca = PCA(n_components = k)
X_new = pca.fit_transform(X_ebo_ml)
y_new = y_ebo_ml.copy()


In [ ]:
def myplot(X_new,y_new,coeff,labels=None):
    xs = X_new[:,0]
    ys = X_new[:,1]
    n = coeff.shape[0]
    cdict = {0: 'gray', 1: 'black'}
    ldict = {0: 'Not a case', 1: 'Confirmed'}
    fig, ax = plt.subplots()
    for g in np.unique(y_new):
        ix = np.where(y_new == g)
        ax.scatter(xs[ix], ys[ix], c = cdict[g], label = ldict[g])
    for i in range(n):
        factor_ = 2.8
        plt.arrow(0, 0, coeff[i,0]*factor_, coeff[i,1]*factor_,color = 'r',alpha = 0.5)
        if labels is None and np.linalg.norm([coeff[i,0], coeff[i,1]])>0.9:
            plt.text(coeff[i,0]* 2, coeff[i,1] * 2, str(X_ebo_ml.columns[i]), color = 'red', ha = 'center', va = 'center', fontsize=12, weight='bold')
    
    ax.legend()
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    plt.grid()
    plt.show()
    
#Call the function. Use only the 2 PCs.
myplot(X_new, y_new, np.transpose(pca.components_[0:2, :]))


Let us apply PCA but dropping the sex, age and referraltime 

In [ ]:
def myplot(X_new,y_new,coeff,labels=None):
    xs = X_new[:,0]
    ys = X_new[:,1]
    n = coeff.shape[0]
    cdict = {0: 'blue', 1: 'pink'}
    ldict = {0: 'Not a case', 1: 'Confirmed'}
    fig, ax = plt.subplots()
    for g in np.unique(y_new):
        ix = np.where(y_new == g)
        ax.scatter(xs[ix], ys[ix], c = cdict[g], label = ldict[g])
    for i in range(n):
        factor_ = 2.8
        plt.arrow(0, 0, coeff[i,0]*factor_, coeff[i,1]*factor_,color = 'r',alpha = 0.5)
        if labels is None and np.linalg.norm([coeff[i,0], coeff[i,1]])>0.3:
            plt.text(coeff[i,0]* 3, coeff[i,1] * 3, str(X_ebo_ml.columns[i]), color = 'black', ha = 'center', va = 'center', fontsize=12, weight='bold')
    
    ax.legend()
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    plt.grid()
    plt.show()

In [ ]:
X_drop = X_ebo_ml.drop(columns=['Sex', 'Age', 'Referraltime'])

# apply PCA 
k = 2
pca = PCA(n_components = k)
X_new = pca.fit_transform(X_drop)
y_new = y_ebo_ml.copy()

#Call the function. Use only the 2 PCs.
myplot(X_new, y_new, np.transpose(pca.components_[0:2, :]))

### Split train/test

Before standardizing, we need to make sure that the dataset is split between train and test !
This is to make sure that the "way" we standardize our train set is "the base" as to how we standardize our test set.

In [ ]:

X_ebo_train, X_ebo_test, y_ebo_train, y_ebo_test = train_test_split(X_ebo_ml, y_ebo_ml, test_size=0.2, random_state=0, stratify=y_ebo_ml)


# Instantiate the visualizer
visualizer = ClassBalance(labels=['Ebola Negative', 'Ebola Positive'])

visualizer.fit(y_ebo_train, y_ebo_test)        # Fit the data to the visualizer
visualizer.show()                      # Finalize and render the figure
plt.show()

##### Normalizing the data


In [ ]:
scaler = StandardScaler()
numerical_col = ['Age','Referraltime']
X_ebo_train.loc[:,numerical_col] = scaler.fit_transform(X_ebo_train[numerical_col])
X_ebo_test.loc[:,numerical_col] = scaler.transform(X_ebo_test[numerical_col])

In [ ]:
df_triage[symptoms]



### Least squares

In [ ]:
import statsmodels.api as sm

X = sm.add_constant(df_triage.loc[:, df_triage.columns != 'FinalStatus'])
y = df_triage['FinalStatus']
#normalize the continuous variables
scaler = StandardScaler()

X[['Age','Referraltime']] = scaler.fit_transform(X[['Age','Referraltime']])

# Model
est_OLS = sm.OLS(y, X.astype(float)).fit()
print(est_OLS.summary())

### Logistic

In [ ]:


est_logit = sm.Logit(y, X.astype(float)).fit()
print(est_logit.summary())

We can see terrible r² values and we'll proceed to do some feature selection in order to improve the model

#### Backward Elimination

- Feed all possible features to the model
- Compute perfomance of model and remove worst perfoming features until stopping criterion



In [ ]:
#### For Least square

X = df_triage.loc[:, df_triage.columns != 'FinalStatus']
y = df_triage['FinalStatus']
tl
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
## Assert the model with new features
X = sm.add_constant(df_triage.loc[:, selected_features_BE])
y = df_triage['FinalStatus']

est_OLS = sm.OLS(y, X.astype(float)).fit()
print(est_OLS.summary())

### Logistic regression Backward elimation

In [ ]:
X = df_triage.loc[:, df_triage.columns != 'FinalStatus']
y = df_triage['FinalStatus']

#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.Logit(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
### Update the model with the selected features 
X = sm.add_constant(df_triage.loc[:, selected_features_BE])
y = df_triage['FinalStatus']

est_Logit = sm.Logit(y, X.astype(float)).fit()
print(est_Logit.summary())

The Pseudo-R^2 indicates that the model is not a good fit. 

#### Recursive Feature Elimination
The Recursive Feature Elimination (RFE) method works by recursively removing attributes and building a model on those attributes that remain. It uses accuracy metric to rank the feature according to their importance. The RFE method takes the model to be used and the number of required features as input. It then gives the ranking of all the variables, 1 being most important. It also gives its support, True being relevant feature and False being irrelevant feature.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

X = df_triage.loc[:, df_triage.columns != 'FinalStatus']
y = df_triage['FinalStatus']

#no of features 
nof_list=np.arange(1, len(X.columns)+1) 

highest_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,n_features_to_select=nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    #mean accuracy
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>highest_score):
        highest_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, highest_score))

In [ ]:
from sklearn.linear_model import LogisticRegression

X = df_triage.loc[:, df_triage.columns != 'FinalStatus']
y = df_triage['FinalStatus']

#no of features 
nof_list=np.arange(1, len(X.columns)+1)   

highest_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LogisticRegression(max_iter = 200)
    rfe = RFE(model,n_features_to_select=nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>highest_score):
        highest_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, highest_score))

In [ ]:
## The RFE computed the number of features we should select (11)

cols = np.array(X.columns)
model = LogisticRegression(max_iter = 200)

#Initializing RFE model
rfe = RFE(model, n_features_to_select=11)   

#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  

#Fitting the data to model
model.fit(X_rfe,y) 
print("R_squared: " + str(model.score(X_rfe, y)))
print(cols[rfe.support_])
print(model.coef_)

#### Ensemble Methods
Embedded methods are a catch-all group of techniques which perform feature selection as part of the model construction process. The exemplar of this approach is the LASSO method for constructing a linear model, which penalizes the regression coefficients with an L1 penalty, shrinking many of them to zero. Any features which have non-zero regression coefficients are 'selected' by the LASSO algorithm.

In [ ]:
from sklearn.linear_model import LassoCV
X = df_triage.loc[:, df_triage.columns != 'FinalStatus']
y = df_triage['FinalStatus']

reg = LassoCV()
reg.fit(X, y)

print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")
plt.show()

# Let us try some more "complex" and easily interpretable models

### Decision Trees
We did a few trees of varying depth. The problem is overfitting, which is fought with random forests. 

In [ ]:
from sklearn import tree

###### If you would like to export the following trees, replace the number preceeding the (denoted by '?') q by your interested tree. Where this number corresponds to the number of questions asked by the doctor 


from sklearn import tree
import graphviz
dot_data_?q = tree.export_graphviz(clf?q, 
                                out_file=None, 
                                feature_names=X.columns,
                                rounded=True)
graph_?q = graphviz.Source(dot_data_?q)
graph_?q.render("Decision_tree_?_q")

##### What happens when you only can ask one question ?

In [ ]:

clf1q = tree.DecisionTreeClassifier(max_depth=1)
clf1q = clf1q.fit(X_ebo_train, y_ebo_train)


In [ ]:
score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, clf1q)

In [ ]:
tree.plot_tree(clf1q,feature_names=X.columns)
fig = matplotlib.pyplot.gcf()

fig.savefig('tree_1_question.png')
plt.show()


In [ ]:
# What happens when you only can ask two question ?

clf2q = tree.DecisionTreeClassifier(max_depth=2)
clf2q = clf2q.fit(X_ebo_train, y_ebo_train)

score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, clf2q)


In [ ]:

tree.plot_tree(clf2q,feature_names=X.columns)
fig = matplotlib.pyplot.gcf()
fig.savefig('tree_2_question.png')
plt.show()


In [ ]:
# Now with ten questions 

clf10q = tree.DecisionTreeClassifier(max_depth=10)
clf10q = clf10q.fit(X_ebo_train, y_ebo_train)

score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, clf10q)


Decision trees are easy to overfit (refer to result just above), let's use Random forest and tune its hyper-parameters

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

# What happens when you only can ask five question ?
clf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state = 123)

clf = clf.fit(X_ebo_train, y_ebo_train)

score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, clf)



#### Hyper-parameter tuning 

In [ ]:
from sklearn.model_selection import GridSearchCV
param_test = {
    'n_estimators':[i for i in range(5, 20)],
    'max_depth':[i for i in range(3, 15)],
    'min_samples_split':[i for i in range(2, 7)]
}
clforest = RandomForestClassifier(random_state = 123)


In [ ]:

gsearch = GridSearchCV(
    estimator= clforest,
    param_grid= param_test,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=5
)
gsearch.fit(X_ebo_train, y_ebo_train)


In [ ]:
gsearch.best_params_

In [ ]:

clf_best = RandomForestClassifier(n_estimators=19, max_depth=9, min_samples_split=4)

clf_best = clf_best.fit(X_ebo_train, y_ebo_train)
score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, clf_best)



Again great signs of overfitting 

Let us add a hyper-parameter which may help with the overfitting 


In [ ]:
parameters = {'n_estimators': [2,3,4,5,6], 'max_depth' : [3,4,5,6, 7, 8],
              'min_samples_split': range(2,6), 'max_leaf_nodes': range(2,5)}

ebo_forest_classifier = RandomForestClassifier()
clf = GridSearchCV(ebo_forest_classifier, parameters, scoring = 'roc_auc', n_jobs=-1)
clf.fit(X_ebo_train, y_ebo_train)

In [ ]:
clf.best_params_

In [ ]:
ebo_forest_classifier = RandomForestClassifier(n_estimators =clf.best_params_['n_estimators'],
                                              max_depth = clf.best_params_['max_depth'],
                                              min_samples_split = clf.best_params_['min_samples_split'],
                                              max_leaf_nodes = clf.best_params_['max_leaf_nodes'])

# Give score to model
score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, ebo_forest_classifier)

Again, the model shows signs of overffing. 

In [ ]:
### This is to  output as a tree the random forest classifier


dot_data_forest =export_graphviz(
    ebo_forest_classifier.estimators_[1],
    out_file=None,
    feature_names=X.columns,
    class_names=['Not a case', 'Confirmed'],
    label='root',
    filled=True,
    rounded=True,
    impurity=False,
    proportion=True
)
graph_forest = graphviz.Source(dot_data_forest)
graph_forest.render("Decision_forest_1")


## XGBoost 

(eXtreme Gradient Boosting) : Advanced implementation of gradient boosting algorithm

__Import libraries__

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from matplotlib.pylab import rcParams
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance


##### __Hyperparameters__
__BOOSTER PARAMETER__
- _learning rate_:         eta = [[0,1]] (Use CV)
- _min_child_weigt_:   (determines sum of weights of all weights required in a child) Controls overfitting, high values prevent models to learn from highly specific samples
(Use CV)
- _max_depth_: determines depth of tree [3-10] (Use CV)
- _subsample_: % of samples used per tree, low value may lead to underfitting [0.5-1]
- _colsample_bytree_: % features used per tree, high value, then overfitting [0.5-1]


__Learning Task Parameters__
- _objective_: (binary:logistic )logistic regression for binary classification, returns predicted probability (not class)
- _eval_metric_= "error" Binary classification 0.5 threshold 
- _n_estimators_: number of trees to built




__To penalize__
- _gamma_: The larger gamma is, the more conservative the algorithm will be. (loss required to split)
- _alpha_: L1 regularization
- _lambda_: L2 reg, smoother than L1


In [ ]:
def modelfit(alg,x_train, y_train, x_test, y_test,useTrainCV=True, cv_folds=5, early_stopping_rounds=100):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(x_train, y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds,show_stdv=False)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(cvresult.shape[0])
    
    #Fit the algorithm on the data
    eval_set = [(x_train, y_train), (x_test, y_test)]
    alg.fit(x_train, y_train,eval_metric='error', eval_set=eval_set, verbose=False)

        
    #Predict training set:
    dtrain_predictions = alg.predict(x_train)
    dtrain_predprob = alg.predict_proba(x_train)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("F1 accuracy score (train): ",metrics.f1_score(y_train,model.predict(x_train)))
    print ("Accuracy (train): %.3f" %metrics.accuracy_score(y_train, dtrain_predictions))
    print("Accuracy (test): %.3f" %metrics.accuracy_score(y_test, alg.predict(x_test)))
    print("F1 accuracy score (test): ",metrics.f1_score(y_test,model.predict(x_test)))
    print("AUC Accuracy (train): %f"%metrics.roc_auc_score(y_train, dtrain_predprob))
    print("AUC Accuracy (test): %f"%metrics.roc_auc_score(y_test, alg.predict_proba(x_test)[:,1]))
 
    # Plot the important features
    xgb.plot_importance(alg)

    
# Code inspired from : https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

## Let's try one by one parameter tuning

General Approach for Parameter Tuning

We will use an approach similar to that of GBM here. The various steps to be performed are:

    Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate. XGBoost has a very useful function called as “cv” which performs cross-validation at each boosting iteration and thus returns the optimum number of trees required.
    
    Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.
    
    Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.
    Lower the learning rate and decide the optimal parameters .


In [ ]:
print("This gives an idea of the scale_pos_weight value  ",sum(y_ebo_train==1)/sum(y_ebo_train==0))

In [ ]:
#############
# tuning n_estimators (number of trees)
#############

model_tune = xgb.XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,
                         gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',
                             nthread=4, seed=123, reg_lambda = 0,scale_pos_weight = 1.78)



param_test1 = {'n_estimators':range(2,20,2)
}
gsearch1 = GridSearchCV(estimator = model_tune, 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_ebo_train,y_ebo_train)
gsearch1.best_params_, gsearch1.best_score_


In [ ]:
# Set parameters
model_tune.set_params(n_estimators=gsearch1.best_params_['n_estimators'])

#### GridSearchCV

Instead of looking at one parameter at a time, let us use GridSearch

In [ ]:
################
# Tune max_depth, et min_child weight
################


param_test1 = {'max_depth':range(3,10,2),'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = 
                        model_tune, 
                                param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_ebo_train,y_ebo_train)
gsearch1.best_params_, gsearch1.best_score_


In [ ]:
# New optimal, let's set those instead 
model_tune.set_params(max_depth=gsearch1.best_params_['max_depth'],min_child_weight=gsearch1.best_params_['min_child_weight'])

In [ ]:
################
# Tune gamma
################


param_test3 = {
 'gamma':[i/10.0 for i in range(0,10)]
}
gsearch3 = GridSearchCV(estimator = model_tune, 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_ebo_train,y_ebo_train)
gsearch3.best_params_, gsearch3.best_score_


In [ ]:
# Set gamma parameter
model_tune.set_params(gamma=gsearch3.best_params_['gamma'])

### Let's have  a look at the model again 
modelfit(model_tune, X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test)
score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, model_tune)

In [ ]:
################
# Tune subsample (% of sample used in trees) and colsample_bytree (% features used by trees)
################

param_test4 = {
 'subsample':[i/10.0 for i in range(5,10)],
 'colsample_bytree':[i/10.0 for i in range(5,10)]
}
gsearch4 = GridSearchCV(estimator =model_tune, 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_ebo_train,y_ebo_train)
gsearch4.best_params_, gsearch4.best_score_



In [ ]:
# Let's try smaller values around the ones found
model_tune.set_params(subsample=gsearch4.best_params_['subsample'],colsample_bytree=gsearch4.best_params_['colsample_bytree'])


In [ ]:
param_test5 = {
 'subsample':[i/100.0 for i in range(70,80,5)],
 'colsample_bytree':[i/100.0 for i in range(65,80,5)]
}
gsearch5 = GridSearchCV(estimator = model_tune, 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_ebo_train,y_ebo_train)
gsearch5.best_params_, gsearch5.best_score_

# It changed a little, we pick these instead

In [ ]:
# Set new parameters "optimal parameters"
model_tune.set_params(subsample=gsearch5.best_params_['subsample'],colsample_bytree=gsearch5.best_params_['colsample_bytree'])

In [ ]:
################
# reg_lambda: L2 reg tuning
################


param_test6 = {
 'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = model_tune, 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch6.fit(X_ebo_train,y_ebo_train)
gsearch6.best_params_, gsearch6.best_score_

In [ ]:
# Let's try tigher values around the one found

param_test7 = {
 'reg_lambda': np.arange(0,1,0.01)
}
gsearch7 = GridSearchCV(estimator = model_tune, 
 param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch7.fit(X_ebo_train,y_ebo_train)
gsearch7.best_params_, gsearch7.best_score_


In [ ]:
model_tune.set_params(reg_lambda=gsearch7.best_params_['reg_lambda'])

In [ ]:
score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, model_tune)

In [ ]:
# Last step is the learning rate




param_test8 = {
 'learning_rate': np.arange(0.01,0.3,0.01)
}
gsearch8 = GridSearchCV(estimator = model_tune, 
 param_grid = param_test8, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch8.fit(X_ebo_train,y_ebo_train)
gsearch8.best_params_, gsearch8.best_score_


In [ ]:
# Set the new learning rate (final parameter)
model_tune.set_params(learning_rate=gsearch8.best_params_['learning_rate'])

# Final model

score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, model_tune)

#### Instead of cross validating, one or two parameters at a time, let's do many 


In [ ]:


param_test = {'learning_rate': np.arange(0.01,0.3,0.1), 'n_estimators':np.arange(9,20,5),'max_depth':[2,3,4], 'gamma': [i/10.0 for i in range(0,3)],
               'min_child_weight':np.arange(5,8,1),'subsample': [i/100.0 for i in range(70,80,5)], 'colsample_bytree': [i/100.0 for i in range(65,80,5)], 
                  'reg_lambda':np.arange(0,7,1)}

gsearch = GridSearchCV(estimator = 
                        XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,
                         gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',
                             nthread=4, seed=123, reg_lambda = 0,scale_pos_weight = 1.78), 
                                param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose = False)
gsearch.fit(X_ebo_train,y_ebo_train)
gsearch.best_params_, gsearch.best_score_



In [ ]:
# Model with optimized set of parameters 
model_CV = XGBClassifier( learning_rate =0.11, n_estimators=19, max_depth=3,
                         min_child_weight=5, gamma=0.1, subsample=0.75, colsample_bytree=0.7,
                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1.78, reg_lambda = 2,seed=123)

# Model report with CV
#modelfit(model_CV, x_train, y_train, x_test, y_test)

In [ ]:
score_model(X_ebo_train, y_ebo_train, X_ebo_test, y_ebo_test, model_CV)

In [ ]:
cumulator.off() 

In [ ]:
cumulator.computation_costs()